# PIPELINE TESTING

In [1]:
#
import os
import numpy as np
import pandas as pd
from pandas import Timestamp
import seaborn as sns
import matplotlib.pyplot as plt

#
from zipline.api import order_target_percent, record, symbol, set_benchmark, get_open_orders
import zipline
import datetime
import pytz
from zipline.data.bundles import load
from trading_calendars import get_calendar
import pyfolio as pf
#
%load_ext zipline

In [2]:
zipline.__version__

'2.2.0'

# Check Symbols of Bundle

In [6]:
trading_calendar=get_calendar('24/7')

now = Timestamp.utcnow()
bundle = load('NFTU', os.environ, now)
symbols = set(str(asset.symbol) 
              for asset in bundle.asset_finder.retrieve_all(
                           bundle.asset_finder.equities_sids))

print(symbols)


{'TFUEL', 'ADA', 'MATIC', 'MANA', 'FLOW', 'SOL', 'AVAX'}


# Simple Strategy

In [7]:
selected_coin = 'ADA'
shortest_period = 7
longest_period = 14
holding_period = 2

def initialize(context):
    context.i = 0
    context.holding_days = 0
    context.bought = False
    set_benchmark(symbol(selected_coin))
    context.asset = symbol(selected_coin)

def handle_data(context, data):
    # skip days for moving average
    context.i += 1
    if context.i < longest_period:
        return
    if context.bought:
        context.holding_days += 1
        
    # trading algorithm
    short_ma = data.history(context.asset, 'close', bar_count=shortest_period, frequency='1d').mean()
    long_ma = data.history(context.asset, 'close', bar_count=longest_period, frequency='1d').mean()

    open_orders = get_open_orders()
    
    # buy signal
    if context.asset not in open_orders:
        if short_ma > long_ma and context.holding_days == 0 and not context.bought:
            order_target_percent(context.asset, 1.0)
            context.bought = True
        
    # sell signal
    if (short_ma < long_ma or context.holding_days == holding_period) and context.bought:
        order_target_percent(context.asset, 0.0)
        context.holding_days = 0
        context.bought = False

    # record price
    record(close=data.current(symbol(selected_coin), 'close'),
          short_ma=short_ma,
          long_ma=long_ma)

In [8]:
# date range MUST BE pd.Timestamp
start_date = pd.Timestamp('2017-11-09', tz='utc')
end_date = pd.Timestamp('2022-04-11', tz='utc')

# performance of algorithm once run
perf = zipline.run_algorithm(start=start_date,
                            end=end_date,
                            trading_calendar=get_calendar('24/7'), 
                            initialize=initialize,
                            capital_base=1000,
                            handle_data=handle_data,
                            bundle='NFTU',
                            data_frequency='daily')
perf.head()
perf.tail()

/Users/dashavasileva/opt/anaconda3/envs/envzipline/lib/python3.8/site-packages/empyrical/stats.py:710: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
/Users/dashavasileva/opt/anaconda3/envs/envzipline/lib/python3.8/site-packages/empyrical/stats.py:799: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)


,period_open,period_close,short_value,returns,long_exposure,pnl,short_exposure,capital_used,orders,transactions,...,sortino,max_drawdown,max_leverage,excess_return,treasury_period_return,trading_days,period_label,close,short_ma,long_ma
2022-04-07 23:59:00+00:00,2022-04-07 00:00:00+00:00,2022-04-07 23:59:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,-0.804715,-0.998902,2.565246,0.0,0.0,1611,2022-04,1.092,1.148000,1.154286
2022-04-08 23:59:00+00:00,2022-04-08 00:00:00+00:00,2022-04-08 23:59:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,-0.804465,-0.998902,2.565246,0.0,0.0,1612,2022-04,1.026,1.128143,1.149143
2022-04-09 23:59:00+00:00,2022-04-09 00:00:00+00:00,2022-04-09 23:59:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,-0.804216,-0.998902,2.565246,0.0,0.0,1613,2022-04,1.045,1.112286,1.141571
2022-04-10 23:59:00+00:00,2022-04-10 00:00:00+00:00,2022-04-10 23:59:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,-0.803967,-0.998902,2.565246,0.0,0.0,1614,2022-04,1.031,1.090143,1.130786
2022-04-11 23:59:00+00:00,2022-04-11 00:00:00+00:00,2022-04-11 23:59:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,-0.803718,-0.998902,2.565246,0.0,0.0,1615,2022-04,0.934,1.050286,1.113929


In [9]:
# Extract algo returns and benchmark returns
returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)
benchmark_period_return = perf['benchmark_period_return']

# Convert benchmark returns to daily returns
#daily_returns = (1 + benchmark_period_return) / (1 + benchmark_period_return.shift()) - 1
daily_benchmark_returns = np.exp(np.log(benchmark_period_return + 1.0).diff()) - 1

# Create tear sheet
pf.create_full_tear_sheet(returns, positions=positions, transactions=transactions, benchmark_rets=daily_benchmark_returns, live_start_date='2021-01-27')

Entire data start date: 2017-11-10
Entire data end date: 2022-04-11
In-sample months: 55
Out-of-sample months: 20


,In-sample,Out-of-sample,All
Annual return,-52.6%,-27.3%,-46.7%
Cumulative returns,-96.9%,-42.7%,-98.2%
Annual volatility,104.7%,47.0%,92.6%
Sharpe ratio,-0.34,-0.45,-0.34
Calmar ratio,-0.53,-0.39,-0.47
Stability,0.92,0.09,0.92
Max drawdown,-99.8%,-69.9%,-99.9%
Omega ratio,0.87,0.87,0.87
Sortino ratio,-0.85,-0.65,-0.80
Skew,13.30,0.80,14.00


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'